In [1]:
from selenium import webdriver
import requests
import re
import nltk
import pandas as pd
from collections import Counter
from datetime import date
from bs4 import BeautifulSoup
from datetime import datetime
from unidecode import unidecode

In [2]:
COUNTRIES = pd.read_csv('pays.csv')
LISTE_COUNTRIES = list(COUNTRIES.Pays)

SITES_EXCLUS = ['wikipedia', 'vikidia','accounts.google','.pdf']
NB_PAGES_GOOGLE = 3

## Fonctions
- Création de la liste d'URL par pays
- Récupération du body des pages
- Nettoyage basique de la page : - html, - caractères spéciaux, - URL, passage en minuscules
- Plusieurs fonctions de génération des dates suivant le besoin

In [3]:
def ListeURL(pays,nresult,DateMin, DateMax):
    Liste_URL = []
    M_Min = str(DateMin.month)
    A_Min = str(DateMin.year)
    M_Max = str(DateMax.month)
    A_Max = str(DateMax.year)
    for x in range(nresult):
        google = driver.get("http://www.google.com/search?q="+pays+"&lr=lang_fr&start="+str(x*10)+"&tbs=cdr%3A1%2Ccd_min%3A"+M_Min+"%2F1%2F"+A_Min+"%2Ccd_max%3A"+M_Max+"%2F11%2F"+A_Max)
        URLs = driver.find_elements_by_xpath("//div[@class='r']/a ")
        URLs = [x.get_attribute('href') for x in URLs]
        URLs = [x for x in URLs if ("wikipedia" not in x) & ("vikidia" not in x) & ("accounts.google" not in x)]
        Liste_URL.extend(URLs)
    return Liste_URL

In [4]:
def getpage(listeURL,num):
    page = requests.get(listeURL[num], timeout = 30).text
    soup = BeautifulSoup(page)
    body = soup.find('body')
    contenu_page = body.get_text()
    return (contenu_page)

In [5]:
def remove_by_regex(txt, regexp):
    return re.sub(regexp, '', txt)

def remove_url(txt):
    return remove_by_regex(txt, re.compile(r"http.?://[^\s]+[\s]?"))

def remove_special_char(txt):
    return re.sub(r"[^a-zA-ZàçéèùïîöôëêäâûüÀÇÉÈÙÏÎÖÔËÊÄÂÛÜœæŒÆ ]", " ", unidecode(txt))

def clean_up(txt):
    txt = remove_url(txt)
    txt = remove_special_char(txt)
    txt = re.sub('<[^<]+?>', '', txt)
    txt = re.sub('\n', ' ', txt)
    txt = re.sub('\t', ' ', txt)
    return txt.lower().strip()


In [6]:
def liste_dates_trim(AnneeMin,AnneeMax):
    MoisMin = ["01-01","01-04","01-07","01-10"]
    MoisMax = ["31-03","30-06","30-09","31-12"]
    Annees = range(AnneeMin,AnneeMax+1)
    liste_date_min = []
    liste_date_max = []
    for x in Annees:
        for y in range(len(MoisMin)):
            liste_date_min.append(MoisMin[y]+'-'+str(x))
            liste_date_max.append(MoisMax[y]+'-'+str(x))
    return liste_date_min, liste_date_max


def liste_dates_an(AnneeMin,AnneeMax):
    Annees = range(AnneeMin,AnneeMax+1)
    liste_date_min = []
    liste_date_max = []
    for x in Annees:
        liste_date_min.append("01-01"+'-'+str(x))
        liste_date_max.append("31-12"+'-'+str(x))
    return liste_date_min, liste_date_max

def liste_dates_ans_separes(Annees):
    liste_date_min = []
    liste_date_max = []
    for x in Annees:
        liste_date_min.append("01-01"+'-'+str(x))
        liste_date_max.append("31-12"+'-'+str(x))
    return liste_date_min, liste_date_max

## initialisation de la base de résultat + ouverture du navigateur

In [7]:
base = pd.DataFrame(columns = ['Date_debut','Date_fin','Pays', 'Rang','Liste_URL', 'Contenu_page'])
driver = webdriver.Chrome(r'C:\Users\lleki\chromedriver.exe')
driver.maximize_window()
dates_min, dates_max = liste_dates_ans_separes([2005])

## Gestions des exceptions
Permet d'ajuster le code pour un périmètre pays et/ou période non prévu (notamment pour compléter une requête interrompue)

In [13]:
#dates_min_select = dates_min[0]
#dates_max_select = dates_max[0]
#dates_min = []
#dates_max = []
#dates_min.append(dates_min_select)
#dates_max.append(dates_max_select)
#
dern_pays = "Ouganda"
rang_der_pays = COUNTRIES[COUNTRIES.Pays == dern_pays]
LISTE_COUNTRIES = LISTE_COUNTRIES[LISTE_COUNTRIES.index(dern_pays)+1:]

## Boucle de constitution de la base
- définit la liste de périodes / de pays sur lesquels requêter
- identifie les URL
- récupère les contenus des pages
- sauvegarde au fur et à mesure
- ferme le navigateur après avoir terminé le process.

In [9]:
for d in range(len(dates_min)):
    DateMin_asdate = datetime.strptime(dates_min[d], '%d-%m-%Y')
    DateMax_asdate = datetime.strptime(dates_max[d], '%d-%m-%Y')
    liste_lignes = []
    sites_bugs = []
    for pays in LISTE_COUNTRIES:
        listeURL = ListeURL(pays, NB_PAGES_GOOGLE,DateMin_asdate, DateMax_asdate)
        for x in range(len(listeURL)):
            try:
                contenu_page = getpage(listeURL,x)
                clean_page = clean_up(contenu_page)
                ligne_base = [DateMin_asdate,DateMax_asdate, pays, x, listeURL[x], clean_page]
                liste_lignes.append(ligne_base)
                pd.DataFrame(liste_lignes).to_csv('lignes_en_cours.csv')
                print(pays+str(x))
            except:
                sites_bugs.append(listeURL[x])
                pd.DataFrame(sites_bugs).to_csv('sites_bugs.csv')
                print("pb sur l'URL" + listeURL[x])
        print(pays+dates_min[d]+"/"+dates_max[d])
        pd.DataFrame(liste_lignes).to_csv('dernier_pays_enregistré.csv')
    base = pd.DataFrame.from_records(liste_lignes)
    base.to_csv('base_résultats_'+str(DateMin_asdate.month)+"-"+str(DateMin_asdate.year)+'_'+str(DateMax_asdate.month)+"-"+str(DateMax_asdate.year)+'.csv')

driver.close()

Afghanistan0
Afghanistan1
Afghanistan2
Afghanistan3
Afghanistan4
Afghanistan5
Afghanistan6
Afghanistan7
Afghanistan8
Afghanistan9
Afghanistan10
Afghanistan11
Afghanistan12
Afghanistan13
Afghanistan14
Afghanistan15
Afghanistan16
Afghanistan17
Afghanistan18
Afghanistan19
Afghanistan20
Afghanistan21
Afghanistan22
Afghanistan23
Afghanistan24
Afghanistan25
Afghanistan26
Afghanistan27
Afghanistan01-01-2005/31-12-2005
